In [13]:
import numpy as np
import cv2

In [27]:
def remove_background(img, threshold):
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, threshed = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

    cnts = cv2.findContours(morphed, 
                            cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)[0]

    cnt = sorted(cnts, key=cv2.contourArea)[-1]

    mask = cv2.drawContours(threshed, cnt, 0, (0, 255, 0), 0)
    masked_data = cv2.bitwise_and(img, img, mask=mask)

    x, y, w, h = cv2.boundingRect(cnt)
    dst = masked_data[y: y + h, x: x + w]

    dst_gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    _, alpha = cv2.threshold(dst_gray, 0, 255, cv2.THRESH_BINARY)
    b, g, r = cv2.split(dst)

    rgba = [r, g, b, alpha]
    dst = cv2.merge(rgba, 4)

    return dst

In [32]:
def vdf(video_name):  # video processing function (영상 처리 함수)
    input_video = video_name
    cap = cv2.VideoCapture(input_video)  # 동영상 캡처 객체 생성
    fps = cap.get(cv2.CAP_PROP_FPS)  # 초당 프레임
    fwt = round((1 / fps) * 1000)  # 프레임당 재생시간 설정 & 입력단위가 ms 이므로 *1000 & 정수입력으로 round 처리

    cbs = cv2.createBackgroundSubtractorKNN()  # 배경제거 객체생성함수 선언

    if cap.isOpened():  # 캡쳐객체 초기화 확인 (동영상을 불러올 수 있는지 확인하는듯)

        while cap.isOpened():  # 초기화 되는동안 영상재생
            ret, frame = cap.read()  # 프레임 추출가능(T/F), 추출한프레임

            if ret:  # run = True (캡쳐 객체 호출가능) : 동영상 재생시작
                origin_frame = cv2.resize(frame, (480, 320))  # 프레임이미지 크기 변환 480*320
                remove_frame = remove_background(origin_frame, 250)

                cv2.imshow(input_video, origin_frame)  # 입력비디오의 프레임 이미지를 보여줌
                cv2.imshow('remove_' + input_video, remove_frame)

                if cv2.waitKey(fwt) == ord('q'):  # fps 만큼 재생하다가 'q' 입력되면 동작 종료
                    cap.release()
                    cv2.destroyAllWindows()
                    print("Stop video")
                    break

            else:  # run = False (캡쳐 객체 호출불가능) : 동영상 자동 종료
                cap.release()  # 캡쳐 자원 반납
                cv2.destroyAllWindows()  # 창종료
                print("End of playback")

    else:
        print("Can't open")

In [33]:
vdf('smp.mp4')

Stop video
